#### This is the code for ground motion visualization following an earthquake.
Authors: Akash Kharita & Raul Mendoza
2021/08/17 


You are free to modify this code on your own and if you get interesting results, please let me know at akharita1999@gmail.com

In [ ]:
# Importing the necessary dependencies. 
from glob import glob 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import pygmt
from obspy.taup import TauPyModel
import time
import csv
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import cartopy.mpl.geoaxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.animation import FuncAnimation
import pandas as pd
import matplotlib as mpl
import cartopy.io.img_tiles as cimgt


Input required is a decimated stream containing traces corresponding to all the stations and should have same number of samples. 

if st is our stream-
st_dec = st.decimate(factor=n)

x_lats = list containing latitudes of all the stations
y_lons = list containing longitudes of all the stations

In [ ]:
%matplotlib inline
st_plot = st_dec.copy()    #st_dec is decimated stream.
new_sampling_rate = 1   #Adjust this after decimating.

#Normalize data from -1 to +1.
#Then add +0.5, so that the range is from -0.5 to 1.5.
#Since the color map assigns color from 0 to 1,
#color will saturate from -0.5 to 0 (25% and lower) and from 1 to 1.5 (75% and higher).
for k in range(len(st_plot)):
    st_plot[k].data = st_plot[k].data/np.max(abs(st_plot[k].data))+0.5

no_of_traces  = len(st_plot)
no_of_samples = len(st_plot[0].data)

color = [(1.0, 1.0, 1.0, 1.0)]*no_of_traces #Create a list of white RGB color
color = no_of_samples*[color]
color = np.array(color) #Convert list to an array

#Set the color map (search matplotlib documentation for available color maps)
cmap = mpl.cm.get_cmap('bwr')

#
for k in range(len(st_plot)): #For each trace...
    for j in range(no_of_samples-1):  #For each sample point...
        #Set the RGB color code based on the normalized value.
        if st_plot[k].data[j] <= 0:
            color[j,k] = cmap(0) #blue
        elif st_plot[k].data[j] >= 1:
            color[j,k] = cmap(1) #red
        else:
            color[j,k] = cmap(st_plot[k].data[j])

#Create a figure.
%matplotlib notebook
fig = plt.figure(figsize=[8,10])
t = 0 #This will be used in the caption to display time.

ax = plt.axes(projection=ccrs.Mercator())
#ax.set_global
ax.set_extent([-140, -170, 45, 70])
ax.coastlines()
#ax.stock_img()
#ax.legend() (doesn't work)

#add background imagery from maps.stamen.com (suggested by Cartopy)
stamen_terrain = cimgt.Stamen('terrain-background')
ax.add_image(stamen_terrain, 5) #2nd argument is the zoom level.

#Define what is to be plotted at each frame.
def animate(i):
    # i will iterate through samples in your trace
    t = i/new_sampling_rate #The 20th sample at a rate of 5 Hz is the (20/5 =) 4th second.
    plt.title('2021-07-29 Mw 8.2 Chignik,Alaska Earthquake\nGround Motion Visualization\nsamples = '+str(t)+'/'+str(no_of_samples))
    plt.scatter(x_lons, y_lats, c=color[i], transform=ccrs.PlateCarree())
    
    #Label the stations. This works, but isn't pretty when stations are close to each other.
    #for m, tr in enumerate(st_plot):
        #label = tr.stats.network + ' ' + tr.stats.station
        #ax.text(x_lons[m]+0.05, y_lats[m], label, transform=ccrs.PlateCarree())
    
    #Plot epicenter.
    if t == 0:
        plt.scatter(longitude,latitude, c='black', marker='*',transform=ccrs.PlateCarree())  #longitude and latitude are the event coordinates
        #ax.text(longitude-1, latitude+.3, 'Mw ' + moment_magnitude, transform=ccrs.PlateCarree())
        ax.text(longitude-0.5, latitude+.2, 'Epicenter', transform=ccrs.PlateCarree())
        
#Call the FuncAnimation function.
output = FuncAnimation(fig, animate, frames = no_of_samples, interval=1,repeat = False,blit=True)

#Show output.
plt.show()

#Save output.
output.save('filename.mp4', writer = 'ffmpeg', fps = 30)